# Enumerations
LinkML supports a wide spectrum of enumerations, from simple option lists to values come directly from
ontologies or pre-defined value sets.

In [1]:
!pip install -q --disable-pip-version-check yamlmagic
%reload_ext yamlmagic

Helper function to compile linkml model to python

In [2]:
from linkml.generators.pythongen import PythonGenerator
from linkml.utils.compile_python import compile_python
from linkml_model.meta import SchemaDefinition

def compile_model(model: str, print_python: bool=False) -> SchemaDefinition:
    gen = PythonGenerator(model, merge_imports=False, gen_classvars=False, gen_slots=False)
    code = gen.serialize()
    if print_python:
        print(code)
    return compile_python(code)

## The simplest form of a LinkML enumeration is a list of tag and (optional) values


In [3]:
%%yaml model

id: http://example.org/test/simple
name: simple
description: Very simple enumeration

prefixes:
  linkml: https://w3id.org/linkml/
  play: http://example.org/test/play/
  CS: http://ontologies-r.us/codesystem/

default_range: string
default_prefix: play


imports:
  - linkml:types


enums:
  Position:
    description: Baseline enumeration -- simple code/value pairs, where the value (description) is optional
    permissible_values:
      a: top
      b: middle
      c: bottom
      d:

classes:
    PositionalRecord:
        description: The combination of a unique identifier and with an associated position
        attributes:
            id:
                range: string
                identifier: true
            position:
                range: Position
                required: true

In [4]:
module = compile_model(model, print_python=True)
# Create a record selecting position "a" (top)
print('-' * 40)
c1 = module.PositionalRecord('my location', 'a')
print(str(c1))
print(str(c1.position))
print(repr(c1.position))
try:
    c2 = module.PositionalRecord('your location', 'z')
except ValueError as e:
    print(e)


## Note that non-python values still work, although they won't appear in the auto-complete section

In [5]:
%%yaml model2

id: http://example.org/test/simple
name: simple
description: Enumeration with some non-std values

prefixes:
  linkml: https://w3id.org/linkml/
  play: http://example.org/test/play/

default_range: string
default_prefix: play

imports:
  - linkml:types

classes:
    Sample:
        attributes:
            key:
                range: string
                identifier: true
            enumentry:
                range: Unusual enum patterns
                multivalued: true

enums:
  Unusual enum patterns:
    description: Very odd enumeration
    permissible_values:
      M: Normal selection
      1: Numeric selection
      def: Python reserved word
      "embedded space": Embedded space
      "% ! -- whoo": Really weird stuff

In [6]:
module = compile_model(model2, print_python=True)
t = module.Sample("Something", [module.UnusualEnumPatterns.M, module.UnusualEnumPatterns['% ! -- whoo']])
print(str(t))

## Enumerations can be "bound" to external URI's
The URI's can be emitted as Curies OR URI's (your choice)

In [7]:
%%yaml model3
id: http://example.org/test/simple
name: simple
description: Very simple enumeration

prefixes:
  linkml: https://w3id.org/linkml/
  play: http://example.org/test/play/
  SCT: http://snomed.info/id/

default_range: string
default_prefix: play

imports:
  - linkml:types

enums:
  Colors:
    description: Color values, mapped to SNOMED CT
    permissible_values:
      1:
        description: Red
        meaning: SCT:371240000
      2:
        description: Yellow
        meaning: SCT:371244009
      3:
        meaning: SCT:405738005
      4:
        description: Muted
        meaning: SCT:abcde
        notes:
            - Note that CURIEs are not validated if a code_set is not specified
      9:
        description: Muddy
        notes:
            - "There isn't a SCT map for this term"

classes:
    FavoriteColor:
        attributes:
            id:
                range: string
                identifier: true
            position:
                range: Colors
                required: true

In [8]:
module = compile_model(model3)
colorrec = module.FavoriteColor("Harold", module.Colors['2'])
print(colorrec)
print(str(colorrec.position))
print(colorrec.position.meaning)
cr2 = module.FavoriteColor("Donald", module.Colors['4'])
print(cr2.position.meaning)

# Enumerations and code sets
Enumerations have additional variables that reference code sets (i.e. a list of URI's)
* `code_set` - A curie that references a list of URI's. Can be:
    * The URI of an entire ontology, which uses the 'CS' prefix (e.g. `CS:SCT`, `CS:HP`, ...)
    * The URI of a code set definition (see: TCCM model)
* `code_set_tag` - the (case insensitive) tag assigned to a particular release of a code set by a terminology service.
   If default tag, if neither a tag or a version is supplied, is "Current", the meaning of which
   is assigned by the service.
* `code_set_version` - a (case sensitive) version identifier that names a specific code set.

In the example above, the URI, `http://snomed.info/id/abcde` was accepted as a valid meaning
association.  If, however, we supply the SNOMED CT Code set (CS:SCT):

1. The enumeration will only accept _valid_ SCT URI's and
2. Any valid SCT URI will automatically provide the description and, if appropriate, its description

*Note: Need to make it clear that CS:HP is actually shorthand for a CodeSetDefinition that states ALL codes from
the CURRENT version of CS:HP.*

*Note 2: Also same thing for descendents of:  SCT:74400008 == NS1:SCTAppendicites(root=SCT:744000008, rel=descendants)*